# Tutorial showing the integration between the QDAC2 and the OPX to perform voltage sweeps using the pyvisa commands

In this tutorial you will learn how to combine the QDAC2 with the OPX to perform n-dimensional voltage scans efficiently.

The trick here is to use a digital marker from the OPX to trigger the QDAC and make it step the output voltage of a given channel across a pre-loaded voltage list. Please note that the size of this list is limited to 65536 items per channel. To this purpose, an element called "qdac_trigger1" was created in the config and a digital pulse was defined. You can read more about digital waveform manipulations here: https://docs.quantum-machines.co/0.1/qm-qua-sdk/docs/Introduction/qua_overview/?h=digi#digital-waveform-manipulations

In this case, the latency induced by the communication between the control PC and the instruments and by transferring the data is suppressed since everything is controlled by the OPX pulse processor. The experimental runtime is then given by the QDAC bandwidth and the integration time. 

In [ ]:
import pyvisa as visa
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from configuration import *
import numpy as np
import matplotlib.pyplot as plt
from time import time, sleep
from qualang_tools.plot import interrupt_on_close
from qualang_tools.results import wait_until_job_is_paused, fetching_tool

%matplotlib qt

## Helper functions

A class called **QDACII** is created to enable the communication with the QDAC. 
It can also be modified to create your own driver.

A function is also defined in order to easily program the QDAC2 to load a voltage list and step through it on the event of an external trigger (provided by the OPX here). Several parameters such as the filter, dynamic range and dwell time can be set.

In [ ]:
# QDAC2 instrument class
class QDACII:
    def __init__(
        self,
        communication_type: str,
        IP_address: str = None,
        port: int = 5025,
        USB_device: int = None,
        lib: str = "@py",
    ):
        """
        Open the communication to a QDAC2 instrument with python. The communication can be enabled via either Ethernet or USB.

        :param communication_type: Can be either "Ethernet" or "USB".
        :param IP_address: IP address of the instrument - required only for Ethernet communication.
        :param port: port of the instrument, 5025 by default - required only for Ethernet communication.
        :param USB_device: identification number of the device - required only for USB communication.
        :param lib: use '@py' to use pyvisa-py backend (default).
        """
        rm = visa.ResourceManager(lib)  # To use pyvisa-py backend, use argument '@py'
        if communication_type == "Ethernet":
            self._visa = rm.open_resource(f"TCPIP::{IP_address}::{port}::SOCKET")
            self._visa.baud_rate = 921600
            # self._visa.send_end = False
        elif communication_type == "USB":
            self._visa = rm.open_resource(f"ASRL{USB_device}::INSTR")

        self._visa.write_termination = "\n"
        self._visa.read_termination = "\n"
        print(self._visa.query("*IDN?"))
        print(self._visa.query("syst:err:all?"))

    def query(self, cmd):
        return self._visa.query(cmd)

    def write(self, cmd):
        self._visa.write(cmd)

    def write_binary_values(self, cmd, values):
        self._visa.write_binary_values(cmd, values)

    def __exit__(self):
        self.close()


# load list of voltages to the relevant QDAC2 channel
def load_voltage_list(
    qdac, channel: int, dwell: float, slew_rate: float, trigger_port: str, output_range: str, output_filter: str, voltage_list: list
):
    """
    Configure a QDAC2 channel to play a set of voltages from a given list and step through it according to an external trigger given by an OPX digital marker, using pyvisa commands.

    :param qdac: the QDAC2 object.
    :param channel: the QDAC2 channel that will output the voltage from the voltage list.
    :param dwell: dwell time at each voltage level in seconds - must be smaller than the trigger spacing and larger than 2e-6.
    :param slew_rate: the rate at which the voltage can change in Volt per seconds to avoid transients from abruptly stepping the voltage. Must be within [0.01; 2e7].
    :param trigger_port: external trigger port to which a digital marker from the OPX is connected - must be in ["ext1", "ext2", "ext3", "ext4"].
    :param output_range: the channel output range that can be either "low" (+/-2V) or "high" (+/-10V).
    :param output_filter: the channel output filter that can be either "dc" (10Hz), "med" (10kHz) or "high" (300kHZ).
    :param voltage_list: list containing the desired voltages to output - the size of the list must not exceed 65536 items.
    :return:
    """
    # Load the list of voltages
    qdac.write_binary_values(f"sour{channel}:dc:list:volt ", voltage_list)
    # Ensure that the output voltage will start from the beginning of the list.
    qdac.write(f"sour{channel}:dc:init:cont off")
    # Set the minimum time spent on each voltage level. Must be between 2µs and the time between two trigger events.
    qdac.write(f"sour{channel}:dc:list:dwell {dwell}")
    # Set the maximum voltage slope in V/s
    qdac.write(f"sour{channel}:dc:volt:slew {slew_rate}")
    # Step through the voltage list on the event of a trigger
    qdac.write(f"sour{channel}:dc:list:tmode stepped")
    # Set the external trigger port. Must be in ["ext1", "ext2", "ext3", "ext4"]
    qdac.write(f"sour{channel}:dc:trig:sour {trigger_port}")
    # Listen continuously to trigger
    qdac.write(f"sour{channel}:dc:init:cont on")
    # Make sure that the correct DC mode (LIST) is set, as opposed to FIXed.
    qdac.write(f"sour{channel}:dc:mode LIST")
    # Set the channel output range
    qdac.write(f"sour{channel}:rang {output_range}")
    # Set the channel output filter
    qdac.write(f"sour{channel}:filt {output_filter}")
    sleep(1)
    print(
        f"Set-up QDAC2 channel {channel} to step voltages from a list of {len(voltage_list)} items on trigger events from the {trigger_port} port with a {qdac.query(f'sour{channel}:dc:list:dwell?')} s dwell time."
    )

## Connect to the instruments

In [ ]:
# Create the qdac instrument
qdac = QDACII("Ethernet", IP_address="127.0.0.1", port=5025)  # Using Ethernet protocol
# qdac = QDACII("USB", USB_device=4)  # Using USB protocol

In [ ]:
# Open a Quantum Machine Manager
qmm = QuantumMachinesManager(host="127.0.0.1", cluster_name="my_cluster")

## Example experiments:
1. 1D voltage sweep performed by the QDAC only
2. 1D voltage sweep performed by the QDAC triggered by the OPX
3. 2D voltage sweep where both axes are scanned using the triggering method

### Experiment 1: 1D voltage sweep performed by the QDAC only
In this example, the QDAC is parametrized to output a constant voltage which is swept outside of QUA using a python loop. Here the OPX is simply measuring at each level, but the QUA program can easily be modified to complexify the sequence.

This method is similar to how any external DC voltage source would be integrated with the OPX without triggering.

In [ ]:
n_avg = 100  # Number of averaging loops
# Voltage values in Volt
voltage_values1 = list(np.linspace(-1.5, 1.5, 101))

### OPX section
with program() as prog:
    i = declare(int)
    n = declare(int)
    data = declare(fixed)
    data_st = declare_stream()
    with for_(i, 0, i < len(voltage_values1), i + 1):
        pause()
        with for_(n, 0, n < n_avg, n + 1):
            # Wait 1ms before measuring (depends of the QDAC filter option)
            wait(1_000_000 // 4, "readout_element")
            # Measure for 10µs with the OPX - Can be replaced by dual_demod, demod or else
            measure("readout", "readout_element", None, integration.full("const", data, "out1"))
            # Send the result to the stream processing
            save(data, data_st)

    with stream_processing():
        # Average all the data and save the values into "data".
        data_st.buffer(n_avg).map(FUNCTIONS.average()).save_all("data")

# Open a quantum machine
qm = qmm.open_qm(config)

### QDAC2 section
qdac_channel = 1
qdac.write("*rst")  # Reset the qdac parameters to start from a blank instrument
# Set the current range ("high" or "low") and filter ("dc": 10Hz ,  "med": 10khz,  "high": 300kHz)
# Set the channel output range
qdac.write(f"sour{qdac_channel}:rang low")
# Set the channel output filter
qdac.write(f"sour{qdac_channel}:filt med")
# Set the slew rate in V/s to avoid transients when abruptly stepping the voltage -Must be within [0.01; 2e7] V/s
qdac.write(f"sour{qdac_channel}:dc:volt:slew {1000}")
qdac.write(f"sour{qdac_channel}:volt:mode fix")

### Run the experiment
start_time = time()
job = qm.execute(prog)
# Live plotting
fig = plt.figure()
interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
data_handle = job.result_handles.get("data")
data_tot = []
for i, vg in enumerate(voltage_values1):
    # Update the QDAC level
    qdac.write(f"sour{qdac_channel}:volt {vg}")
    # Resume the QUA program (escape the 'pause' statement)
    job.resume()
    # Wait until the program reaches the 'pause' statement again, indicating that the QUA program is done
    wait_until_job_is_paused(job)
    # Wait until the data of this run is processed by the stream processing
    data_handle.wait_for_values(i + 1)
    # Fetch the data from the last OPX run corresponding to the current vg
    data = data_handle.fetch(i)["value"] * 2**12 / readout_len
    # Update the list of global results
    data_tot.append(data)
    # Plot results
    plt.cla()
    plt.plot(voltage_values1[:i], data_tot[:i])
    plt.xlabel("QDAC level [V]")
    plt.ylabel("Data [V]")
    plt.pause(0.1)

print(f"Elapsed time: {time() - start_time:.2f} s")
# Interrupt the FPGA program
job.halt()
# Set the QDAC output voltage back to 0
qdac.write(f"sour{qdac_channel}:volt {0}")

### Experiment 2: 1D voltage sweep performed by the QDAC triggered by the OPX

In this example, the QDAC is parametrized to output the values defined in a pre-loaded voltage list. Stepping to the next value is done on the event of a digital marker sent by the OPX to one of the QDAC external trigger input. 
Here the OPX is simply triggering the QDAC and measuring at each level, but the QUA program can easily be modified to complexify the sequence.

This method is very similar to the previous one but significantly reduces the runtime since it avoids the communication time. The speed can also be further improved by removing the live plotting and increasing the QDAC bandwidth.

In [ ]:
n_avg = 100  # Number of averaging loops
# Voltage values in Volt
voltage_values1 = list(np.linspace(-1.5, 1.5, 101))

### OPX section
with program() as qdac_1d_sweep:
    i = declare(int)
    n = declare(int)
    data = declare(fixed)
    data_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        with for_(i, 0, i < len(voltage_values1), i + 1):
            # Wait 1ms before measuring (depends of the QDAC filter option)
            wait(1_000_000 // 4, "qdac_trigger1", "readout_element")
            # Trigger the QDAC channel to output the next voltage level from the list
            play("trig", "qdac_trigger1")
            # Measure with the OPX
            measure("readout", "readout_element", None, integration.full("const", data, "out1"))
            # Send the result to the stream processing
            save(data, data_st)

    with stream_processing():
        # Average all the data and save the values into "data".
        data_st.buffer(len(voltage_values1)).average().save("data")

# Open a quantum machine
qm = qmm.open_qm(config)

### QDAC2 section
qdac.write("*rst")  # Reset the qdac parameters to start from a blank instrument
# Set up the qdac and load the voltage list
load_voltage_list(
    qdac,
    channel=1,
    dwell=2e-6,
    slew_rate=1e3,
    trigger_port="ext1",
    output_range="low",
    output_filter="med",
    voltage_list=voltage_values1,
)

start_time = time()

# Execute the sequence
job = qm.execute(qdac_1d_sweep)
results = fetching_tool(job, ["data"], mode="live")

# Live plotting
fig = plt.figure()
interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
while results.is_processing():
    # Fetch the data
    data = results.fetch_all()[0] * readout_len / 2**12
    # Plot the data
    plt.cla()
    plt.plot(voltage_values1, data * 1000)
    plt.xlabel("QDAC voltage [V]")
    plt.ylabel("data [mV]")
    plt.pause(0.1)

print(f"Elapsed time: {time() - start_time:.2f} s")
qdac.write(f"sour1:volt {0}")

### Experiment 3: 2D voltage sweep where both axes are scanned using the triggering method

In this example, two channels of the QDAC are parametrized to step though two pre-loaded voltage lists on the event of two digital markers provided by the OPX (connected to ext1 and ext2). This method allows the fast acquisition of a 2D voltage map and the data can be fetched from the OPX in real time to enable live plotting (this assumes that the averaging is performed on the most outer loop).

The speed can also be further improved by removing the live plotting and increasing the QDAC bandwidth.

In [ ]:
n_avg = 100  # Number of averaging loops

# Voltage values in Volt
voltage_values1 = np.linspace(-0.4, 0.4, 51)
voltage_values2 = list(np.linspace(-2.5, 2.5, 21))

### OPX section
with program() as qdac_2d_sweep:
    i = declare(int)
    j = declare(int)
    n = declare(int)
    data = declare(fixed)
    data_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        with for_(i, 0, i < len(voltage_values2), i + 1):
            # Trigger the QDAC channel to output the next voltage level from the list
            play("trig", "qdac_trigger2")
            with for_(j, 0, j < len(voltage_values1), j + 1):
                # Trigger the QDAC channel to output the next voltage level from the list
                play("trig", "qdac_trigger1")
                # Wait 1ms before measuring (depends of the QDAC filter option)
                wait(1_000_000 // 4, "readout_element")
                # Measure with the OPX
                measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                # Send the result to the stream processing
                save(data, data_st)

    with stream_processing():
        # Average all the data and save the values into "data".
        data_st.buffer(len(voltage_values1)).buffer(len(voltage_values2)).average().save("data")

# Open a quantum machine
qm = qmm.open_qm(config)

### QDAC2 section
qdac.write("*rst")  # Reset the qdac parameters to start from a blank instrument
# Set up the qdac and load the voltage list
load_voltage_list(
    qdac,
    channel=1,
    dwell=2e-6,
    slew_rate=1e3,
    trigger_port="ext1",
    output_range="low",
    output_filter="med",
    voltage_list=voltage_values1,
)
load_voltage_list(
    qdac,
    channel=2,
    dwell=2e-6,
    slew_rate=1e3,
    trigger_port="ext2",
    output_range="high",
    output_filter="med",
    voltage_list=voltage_values2,
)

start_time = time()

# Execute the sequence
job = qm.execute(qdac_2d_sweep)
results = fetching_tool(job, ["data"], mode="live")

# Live plotting
fig = plt.figure()
interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
while results.is_processing():
    # Fetch the data
    data = results.fetch_all()[0] * readout_len / 2**12
    # Plot the data
    plt.cla()
    plt.pcolor(voltage_values1, voltage_values2, data * 1000)
    plt.xlabel("QDAC channel 1 [V]")
    plt.ylabel("QDAC channel 2 [V]")
    plt.title("data [mV]")
    plt.pause(0.1)

print(f"Elapsed time: {time() - start_time:.2f} s")
qdac.write(f"sour1:volt {0}")
qdac.write(f"sour2:volt {0}")